In [195]:
import pandas as pd
import numpy as np

In [ ]:
bank_df = pd.read_excel("bank_deposit.xlsx")
bank_df

In [ ]:
# Dropped the rows that contained NAN in the contact column.
bank_df= bank_df[bank_df['contact'].notna()]
bank_df

In [ ]:
def quantile_25(bank_df:pd.Series):
    return bank_df.quantile(.25)

def quantile_75(bank_df:pd.Series):
    return bank_df.quantile(.75)

In [ ]:
def replace_w_median(bank_df):
    # find median for columns duration, campaign, pdays, and replace with mode for month
    bank_df['duration'].fillna(bank_df['duration'].median(), inplace=True)
    bank_df['campaign'].fillna(bank_df['campaign'].median(), inplace=True)
    bank_df['pdays'].fillna(bank_df['pdays'].median(), inplace=True)
    bank_df['month'] = bank_df['month'].fillna(bank_df['month'].mode()[0])

In [ ]:
replace_w_median(bank_df)

In [ ]:
bank_df[bank_df.isna().any(axis=1)]

In [ ]:
bank_df.isna().sum()

In [ ]:
bank_df.describe()

In [ ]:
bank_pivot=bank_df.pivot_table(values='balance', index=['Bank deposit(target)','loan'], aggfunc=[ quantile_25, np.median, quantile_75,np.max])
bank_pivot

In [264]:
# Droping customer index 38 because it is a potential duplicate with corrupted "duration" value.
bank_df = bank_df.drop(index=38)

In [243]:
#Median1 is the value for Bank deposit=no, loan=no
median1=int(bank_pivot.loc[(["no"],["no"]),(["median"],["balance"])].values[0])
#Median2 is the value for Bank deposit=yes, loan=no
median2=int(bank_pivot.loc[(["yes"],["no"]),(["median"],["balance"])].values[0])

#These filters allows us to check for multiple conditions and change the "loan" column according to these conditions with a .loc method afterwards.
filter1 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="no") & (bank_df['balance'] > median1)
bank_df.loc[filter1, "loan"] = "no"
filter2 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="no") & (bank_df['balance'] < median1)
bank_df.loc[filter2, "loan"] = "yes"
filter3 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="yes") & (bank_df['balance'] > median2)
bank_df.loc[filter3, "loan"] = "no"
filter4 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="yes") & (bank_df['balance'] < median2)
bank_df.loc[filter4, "loan"] = "yes"

In [ ]:
#Changed NAN into unknown in the "education" column.
bank_df["education"] = bank_df["education"].fillna("unknown")

In [ ]:
#This filter checks if Bank deposit(target) is yes and changes the poutcome to success if the condition is met.
filter_bank_deposit_yes = bank_df["Bank deposit(target)"] == "yes"
bank_df.loc[filter_bank_deposit_yes, "poutcome"] = "success"

In [ ]:
#Checking for duplicate rows
bank_df[bank_df.duplicated(keep=False)].sort_values(by=['balance'], ascending = True)

In [ ]:
# Drop all duplicates in the DataFrame
bank_df = bank_df.drop_duplicates()
bank_df

In [ ]:
#TO CHECK FOR OUTLIERS
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

bank_df.head(100)